In [1]:
import numpy as np
import pandas as pd

In [2]:
DATA_LOC = './data/training.csv'
df = pd.read_csv(DATA_LOC)

In [3]:
def order_to_time(order):
    '''we dont actually need day, just timestamp enough'''
    order = order % 96
    return order//4, order%4 * 15

def format_order(row):
    return row['day']*96 + get_timestamp_order(row['timestamp'])

def get_timestamp_order(ts):
    ts = ts.split(':')
    return int(ts[0])*4 + int(ts[1])//15

def timestamp_to_hour_minute(timestamp):
    timestamp = timestamp.split(':')
    return int(timestamp[0]), int(timestamp[1])

In [4]:
df['order'] = df.apply(format_order, axis=1)

In [16]:
formatted_dict = {}
GEOHASH_LIST = df.geohash6.unique()
MAX_ORDER = np.max(df.order.unique())
for i, gh in enumerate(GEOHASH_LIST):
    temp_df = df[df.geohash6 == gh]
    temp_df.set_index('order', inplace=True)
    temp_dict = temp_df.to_dict('index')
    for order in range(MAX_ORDER):
        hour, minute = order_to_time(order)
        day = order // 96
        if order not in temp_dict:
            demand = 0.0
        else:
            demand = temp_dict[order]['demand']
        formatted_dict[order + i*MAX_ORDER] = {
            'order': order,
            'geohash6': gh,
            'timestamp': '{0}:{1}'.format(hour, minute),
            'day': day,
            'demand': demand,
        }

In [17]:
df = pd.DataFrame.from_dict(formatted_dict, orient='index')
df.head()

,order,geohash6,timestamp,day,demand
0,0,qp03wc,0:0,0,0.054858
1,1,qp03wc,0:15,0,0.086209
2,2,qp03wc,0:30,0,0.050739
3,3,qp03wc,0:45,0,0.075174
4,4,qp03wc,1:0,0,0.062867


In [ ]:
train_df = df[df.day <= 50]
eval_df = df[df.day > 50]

In [32]:
train_df.to_pickle('./data/train_df.pkl')
eval_df.to_pickle('./data/eval_df.pkl')
df.to_pickle('./data/df.pkl')